# Exploratory Data Analysis Workshop
## The Bridge Between Raw Data and Meaningful Insight

**Dataset:** Respiratory Patient Data (OMOP CDM Synthetic Extract)

---

### Learning Objectives

By the end of this workshop, you will be able to:
1. Distinguish between seven types of analytical questions and understand their EDA requirements
2. Apply the four dimensions of exploration (distributional, relational, structural, comparative)
3. Identify data quality issues, patterns, and limitations before modeling
4. Interpret findings and distinguish signal from artifact
5. Determine appropriate next steps based on EDA outcomes

---
## Setup and Imports

Run the cell below to import the required libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-whitegrid')

sns.set(style="whitegrid", context="talk")
plt.rcParams["figure.figsize"] = (10, 6)

%matplotlib inline

# Initialize random number generator for reproducibility, Panda's utilizes NumPy module not "random"
np.random.seed(42)

print("Libraries loaded successfully!")

---
## Part 1: Data Loading and Initial Inspection

### About the Dataset

This dataset contains synthetic patient records extracted from an OMOP Common Data Model (from OHDSI). The records represent patients with respiratory observations, including:
- Patient demographics (age, gender, race, ethnicity)
- Visit information (dates, type, conditions)
- Vital signs (temperature, oxygen saturation, heart rate, etc.)
- Vaccination history
- Patient outcomes

## Intial Analysis
With any dataset, you'll want to perform an initial "exploratory" data analysis to help you understand the structure, patterns, and relationships.

A few goals:
1. **Data Summarization** - gain an quick overview of the dataset
   - **Shape and size of data:** Number of rows, columns, and unique values.
   - **Descriptive statistics:** Mean, median, standard deviation, percentiles.
2. **Data Cleaning** - ensure data quality
   - **Handling missing values:** Identify and impute (mean/median/mode) or remove missing entries.
   - **Removing duplicates:** Eliminate redundant rows or records.
   - **Correcting data types:** Convert data to appropriate formats (e.g., dates, numbers, categories).
   - **Dealing with outliers:** Detect and decide whether to remove or transform extreme values.
3. **Feature Engineering - add additional features/variables to support analysis

In [ ]:
# Read in Files and Establish Starting Dataframes
filename = 'data.csv'

In [ ]:
# Load data
df = pd.read_csv(filename)

In [ ]:
# Perform basic exploratory data analysis
# df.head(n)  # top n rows, n defaults to 5
# df.tail(n)  # last n rows
# df.sample(5) # sample x rows
df

In [ ]:
print("Dataframe shape:",df.shape)
print(df.info())

In [ ]:
print("Dataframe shape:",df.shape)
print(df.info())

In [ ]:
# Convert date fields
for c in ["visit_start_date", "visit_end_date", "birth_datetime", "measurement_Date","flu_last_administered","tdap_last_administered","mmr_last_administered","polio_last_administered"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

In [ ]:
# Examine missing fields
n_rows = len(df)

missing_table = (   # create a new dataframe
    df.isna()
      .agg(['sum', 'mean'])
      .T
      .rename(columns={'sum': 'missing_count', 'mean': 'missing_percent'})
)

missing_table['missing_percent'] = (missing_table['missing_percent'] * 100).round(2)
missing_table['non_missing_count'] = n_rows - missing_table['missing_count']
missing_table['dtype'] = df.dtypes.astype(str)

missing_table = (
    missing_table
      .reset_index(names='column')
      .sort_values(by=['missing_percent', 'column'], ascending=[False, True])
      .set_index('column')
)

missing_table

**Question:** What patterns do you notice in the missing data? Which columns have the most missing values, and why might that be?

In [ ]:
# Create a column for visit length - ignoring visit type
los = (df["visit_end_date"] - df["visit_start_date"]).dt.days
df["length_of_stay_days"] = los.clip(lower=0)


# Modify labels for deceased column
df["deceased_flag"] = df["deceased"].map({"Y": "Deceased", "N": "Alive"}).fillna("Unknown").astype("category")

# columns for year and month
df["visit_year"] = df["visit_start_date"].dt.year
df["visit_month"] = df["visit_start_date"].dt.to_period("M").astype(str)

df['gender_source_value'] = df['gender_source_value'].astype('category')
df['race_source_value'] = df['race_source_value'].astype('category')
df['ethnicity_source_value'] = df['ethnicity_source_value'].astype('category')

In [ ]:
# Descriptive Statistics for Numeric Columns
df.describe()

In [ ]:
# for categorical columns
df.describe(include=['object','category'])

In [ ]:
# Create an alternate view of the conditions, placingin into a separate tidy dataframe
import re

# robust split on ":" allowing extra spaces; keep NaN if empty
def split_conditions(s):
    if pd.isna(s) or str(s).strip() == "":
        return []
    # split on ":" with optional surrounding spaces
    parts = re.split(r"\s*:\s*", str(s))
    # normalize: strip, drop empties, lower (or title-case if you prefer)
    parts = [p.strip() for p in parts if p and p.strip()]
    return parts

# apply once to create a list-typed column
df["condition_list"] = df["condition"].map(split_conditions)

cond_long = (
    df[["visit_occurrence_id", "person_id", "visit_start_date"]]
      .assign(condition_item=df["condition_list"])
      .explode("condition_item", ignore_index=True)
)

# drop rows where no condition exists after cleaning
cond_long = cond_long.dropna(subset=["condition_item"])

# (optional) dedupe within visit in case the same condition appears twice
cond_long = cond_long.drop_duplicates(subset=["visit_occurrence_id", "condition_item"])
cond_long

In [ ]:
# For outpatient visits, assume this is a data issue and the length should be 0
is_outpatient = df['visit_type'].astype(str).str.contains('outpatient', case=False, na=False)

# align dates, then recompute LOS as zero
df.loc[is_outpatient, 'visit_end_date'] = df.loc[is_outpatient, 'visit_start_date']
df.loc[is_outpatient, 'length_of_stay_days'] = 0

# remove any records where length_of_stay_days > 100
df = df[df["length_of_stay_days"] <= 100].copy()

---
## Part 2: Distributional Exploration

**Goal:** Examine individual variables to understand their scale, shape, and validity.

### The Seven Question Types

Before we explore, remember that the type of question determines the EDA approach:

| Type | Core Question | Example |
|------|--------------|--------|
| Descriptive | What happened? | What is the distribution of conditions? |
| Exploratory | What patterns exist? | Is there a relationship between temperature and O2 sat? |
| Inferential | Does this generalize? | Is the O2 sat difference statistically significant? |
| Predictive | What will happen? | Can we predict inpatient admission? |
| Prescriptive | What should we do? | What thresholds should trigger escalation? |
| Causal | What if we intervene? | Would earlier vaccination reduce severity? |
| Mechanistic | What process produces this? | How does symptom progression unfold? |

### Exercise 2.1: Demographic Distributions

Create frequency counts for the following categorical variables:
1. Gender (`gender_source_value`)
2. Race (`race_source_value`)
3. Ethnicity (`ethnicity_source_value`)

**Note:** For demographics, we should look at unique patients, not all visits (since one patient can have multiple visits).

In [ ]:
# First, create a dataframe of unique patients
# YOUR CODE HERE
patients = ...

print(f"Total unique patients: {len(patients)}")

In [ ]:
# Gender distribution
# YOUR CODE HERE


In [ ]:
# Race distribution
# YOUR CODE HERE


In [ ]:
# Ethnicity distribution
# YOUR CODE HERE


### Exercise 2.2: Age Distribution

1. Calculate descriptive statistics for `age_at_visit_years`
2. Create a histogram of the age distribution
3. Create age groups (Pediatric: 0-18, Young Adult: 18-40, Middle Age: 40-65, Elderly: 65+)

In [ ]:
# 1. Descriptive statistics for age
# YOUR CODE HERE


In [ ]:
# 2. Histogram of age distribution
# YOUR CODE HERE


In [ ]:
# 3. Create age groups and show their distribution
# YOUR CODE HERE
# Hint: Use pd.cut() with bins=[0, 18, 40, 65, 100]


### Exercise 2.3: Condition Analysis

The `condition` column contains multiple conditions separated by colons (`:`).

1. Parse the conditions into individual items
2. Count the frequency of each condition
3. Create a bar chart of the top 10 conditions

In [ ]:
# 1. Parse conditions (split by colon)
# YOUR CODE HERE
df['condition_list'] = ...

# 2. Explode and count
all_conditions = ...
condition_counts = ...

In [ ]:
# 3. Bar chart of top 10 conditions
# YOUR CODE HERE


### Exercise 2.4: Vital Signs Distribution (COVID-Suspected Patients)

Many vital signs are only recorded for COVID-suspected visits.

1. Filter the data to only COVID-suspected patients (`observation_source == 'Suspected COVID-19'`)
2. Calculate descriptive statistics for the vital signs columns
3. Create histograms for oxygen saturation, respiratory rate, heart rate, and body temperature

In [ ]:
# 1. Filter to COVID-suspected patients
# YOUR CODE HERE
covid_df = df[df['observation_source'] == 'Suspected COVID-19']

print(f"COVID-suspected visits: {len(covid_df)}")
print(f"Total visits: {len(df)}")
print(f"Percentage: {len(covid_df)/len(df)*100:.1f}%")

In [ ]:
# 2. Descriptive statistics for vital signs
vital_cols = ['oxygen_saturation_percent', 'respiratory_rate_per_minute',
              'heart_rate_bpm', 'body_temperature_c', 'systolic', 'diastolic']

# YOUR CODE HERE


In [ ]:
# 3. Create histograms (2x2 subplot)
# YOUR CODE HERE


**Question:** Are there any concerning values in the vital signs? What might explain extreme values?

*Your answer here:*



### Exercise 2.5: Visit Type Distribution

1. Count the number of Inpatient vs Outpatient visits
2. Calculate the percentage of each type
3. Create a pie chart or bar chart showing the distribution

In [ ]:
# YOUR CODE HERE


### Exercise 2.6: Outlier Detection in Vital Signs

Identify potential outliers in vital sign measurements. Consider clinical validity:
- Temperature: Normal 36-37.5°C, fever >38°C, extreme >42°C
- Oxygen saturation: Normal >95%, concerning <90%, critical <80%
- Heart rate: Normal 60-100 bpm, tachycardia >100, extreme >180
- Respiratory rate: Normal 12-20/min, elevated >24, extreme >40

**Dimension focus:** Distributional exploration

---
## Part 3: Relational Exploration

**Goal:** Investigate how multiple variables interact rather than treating them in isolation.

### Exercise 3.1: Correlation Matrix

Create a correlation matrix heatmap for the numeric vital sign variables using the COVID-suspected patient data.

In [ ]:
# Select numeric columns for correlation
numeric_cols = ['age_at_visit_years', 'oxygen_saturation_percent',
                'respiratory_rate_per_minute', 'heart_rate_bpm',
                'body_temperature_c', 'systolic', 'diastolic']

# YOUR CODE HERE
# Hint: Use covid_df[numeric_cols].corr() and sns.heatmap()


**Question:** What correlations do you observe? Are any surprising or concerning?

*Your answer here:*



### Exercise 3.2: Temperature vs Oxygen Saturation

Create a scatter plot examining the relationship between body temperature and oxygen saturation. Color the points by the `deceased` status.

In [ ]:
# YOUR CODE HERE
# Hint: Use plt.scatter() with c parameter for color, or sns.scatterplot() with hue


### Exercise 3.3: Condition Count and Vital Signs

1. Create a variable `condition_count` that counts the number of conditions per visit (count the colons + 1)
2. Compare the mean vital signs between patients with 1-2 conditions vs 3+ conditions

In [ ]:
# 1. Create condition count
# YOUR CODE HERE
df['condition_count'] = ...

# Check the distribution
df['condition_count'].value_counts().sort_index()

In [ ]:
# 2. Compare vital signs by condition severity (1-2 vs 3+)
# First add condition_count to covid_df
covid_df['condition_count'] = covid_df['condition'].str.count(':') + 1
covid_df['high_condition_count'] = covid_df['condition_count'] >= 3

# YOUR CODE HERE - group by high_condition_count and calculate mean vital signs


### Exercise 3.4: Visit Type and Vital Signs

Compare vital signs between Inpatient and Outpatient visits. Create box plots showing the distribution of oxygen saturation by visit type.

In [ ]:
# YOUR CODE HERE


### Exercise 3.5: Deceased Status and Vital Signs

1. Calculate the mean and median vital signs grouped by deceased status (Y/N)
2. Create box plots comparing oxygen saturation between deceased and non-deceased patients

In [ ]:
# 1. Mean/median vital signs by deceased status
# YOUR CODE HERE


In [ ]:
# 2. Box plots for oxygen saturation by deceased status
# YOUR CODE HERE


---
## Part 4: Structural Exploration

**Goal:** Analyze temporal, hierarchical, and sequential patterns in the data.

### Exercise 4.1: Date Preparation

Convert the date columns to datetime format and extract useful components.

In [ ]:
# Convert date columns to datetime
# YOUR CODE HERE
df['visit_start_date'] = ...
df['visit_end_date'] = ...

# Extract year and month
df['visit_year'] = ...
df['visit_month'] = ...

### Exercise 4.2: Temporal Distribution of Visits

1. Count the number of visits by year
2. For 2020, create a bar chart showing visits by month
3. What patterns do you observe?

In [ ]:
# 1. Visits by year
# YOUR CODE HERE


In [ ]:
# 2. Bar chart of 2020 visits by month
# YOUR CODE HERE


### Exercise 4.3: Length of Stay Analysis

1. Calculate the length of stay (in days) for each visit
2. Filter to inpatient visits only
3. Calculate descriptive statistics for length of stay
4. Create a histogram of length of stay

In [ ]:
# 1. Calculate length of stay
# YOUR CODE HERE
df['length_of_stay'] = ...

In [ ]:
# 2-4. Inpatient length of stay analysis
# YOUR CODE HERE


### Exercise 4.4: Patient Visit History

1. Count the number of visits per patient
2. What percentage of patients have multiple visits?
3. Find a patient with multiple visits and examine their visit history

In [ ]:
# 1. Visits per patient
# YOUR CODE HERE
visits_per_patient = ...

In [ ]:
# 2. Percentage with multiple visits
# YOUR CODE HERE


In [ ]:
# 3. Examine a patient with multiple visits
# YOUR CODE HERE
# Hint: Find the patient with the most visits using idxmax()


### Exercise 4.5: Vaccination Timeline Analysis

1. Calculate the time between the last flu vaccination and the visit date
2. What percentage of patients were vaccinated within the past year (365 days) before their visit?

In [ ]:
# YOUR CODE HERE
df['flu_last_administered'] = pd.to_datetime(df['flu_last_administered'])
df['days_since_flu_vaccine'] = ...

In [ ]:
# Percentage vaccinated within past year
# YOUR CODE HERE


---
## Part 5: Comparative Exploration

**Goal:** Study differences across groups, time periods, or conditions.

### Exercise 5.1: Mortality Rate Comparison

1. Calculate the overall mortality rate (percentage with deceased='Y')
2. Compare mortality rates by visit type (Inpatient vs Outpatient)
3. Compare mortality rates by age group

In [ ]:
# 1. Overall mortality rate
overall_mortality_rate = (df['deceased'] == 'Y').mean() * 100
print(f"Overall mortality rate: {overall_mortality_rate:.2f}%")


In [ ]:
# 2. Mortality by visit type
# YOUR CODE HERE
# Hint: Use pd.crosstab() with normalize='index'
mortality_by_visit = pd.crosstab(
    df['visit_type'],
    df['deceased'],
    normalize='index'
) * 100

mortality_by_visit



In [ ]:
# 3. Mortality by age group
# First ensure age_group exists
if 'age_group' not in df.columns:
    df['age_group'] = pd.cut(df['age_at_visit_years'],
                             bins=[0, 18, 40, 65, 100],
                             labels=['Pediatric', 'Young Adult', 'Middle Age', 'Elderly'])

# YOUR CODE HERE
mortality_by_age = pd.crosstab(
    df['age_group'],
    df['deceased'],
    normalize='index'
) * 100

mortality_by_age


### Exercise 5.2: Visit Type by Age Group

Create a cross-tabulation showing what percentage of each age group has Inpatient vs Outpatient visits.

In [ ]:
# YOUR CODE HERE
visit_by_age = pd.crosstab(
    df['age_group'],
    df['visit_type'],
    normalize='index'
) * 100

visit_by_age


### Exercise 5.3: COVID vs Non-COVID Comparison

Compare characteristics between COVID-suspected and non-COVID visits:
1. Average age
2. Visit type distribution
3. Mortality rate

In [ ]:
# Create a COVID indicator
df['is_covid_suspected'] = df['observation_source'] == 'Suspected COVID-19'

# YOUR CODE HERE - compare the groups
covid_summary = df.groupby('is_covid_suspected').agg(
    avg_age=('age_at_visit_years', 'mean'),
    mortality_rate=('deceased', lambda x: (x == 'Y').mean() * 100),
    inpatient_rate=('visit_type', lambda x: (x.str.contains('Inpatient')).mean() * 100),
    n_visits=('person_id', 'count')
)

covid_summary


### Exercise 5.4: Statistical Significance Testing

Test whether the difference in oxygen saturation between deceased and non-deceased patients is statistically significant.

1. State your null hypothesis
2. Perform a Mann-Whitney U test (non-parametric alternative to t-test)
3. Interpret the results

In [ ]:
# Separate the two groups
deceased_o2 = covid_df[covid_df['deceased'] == 'Y']['oxygen_saturation_percent'].dropna()
survived_o2 = covid_df[covid_df['deceased'] == 'N']['oxygen_saturation_percent'].dropna()

print(f"Deceased - Mean: {deceased_o2.mean():.2f}, Median: {deceased_o2.median():.2f}, n={len(deceased_o2)}")
print(f"Survived - Mean: {survived_o2.mean():.2f}, Median: {survived_o2.median():.2f}, n={len(survived_o2)}")

In [ ]:
# Perform Mann-Whitney U test
# YOUR CODE HERE
# Hint: Use stats.mannwhitneyu()
from scipy import stats

u_stat, p_value = stats.mannwhitneyu(
    deceased_o2,
    survived_o2,
    alternative='two-sided'
)

print(f"Mann–Whitney U statistic: {u_stat:.2f}")
print(f"P-value: {p_value:.4e}")


**Interpretation:** Based on the p-value, is the difference statistically significant at α = 0.05?

The p-value is less than 0.05, indicating a statistically significant difference in oxygen saturation
between deceased and non-deceased patients.



### Exercise 5.5: Condition-Specific Analysis

1. Identify which specific conditions are most associated with inpatient admission
2. Calculate the inpatient rate for each condition
3. Which conditions have the highest inpatient rates?

In [ ]:
# YOUR CODE HERE
# Hint: Explode the condition_list, merge with visit_type, then calculate rates
# Merge condition-level data with visit type
import re

cond_rates = (
    df
    .assign(
        condition_item=df['condition'].str.split(r"\s*;\s*")
    )
    .explode('condition_item')
    .dropna(subset=['condition_item'])
    .assign(
        is_inpatient=lambda x: x['visit_type'].str.contains('Inpatient', na=False)
    )
    .groupby('condition_item')
    .agg(
        inpatient_rate=('is_inpatient', 'mean'),
        total_visits=('visit_occurrence_id', 'count')
    )
    .reset_index()
)

cond_rates['inpatient_rate'] *= 100
cond_rates = cond_rates[cond_rates['total_visits'] >= 100]

cond_rates.sort_values('inpatient_rate', ascending=False).head(10)


---
## Part 6: Interpretation and Synthesis

**Goal:** Move from patterns to meaningful insights.

### Exercise 6.1: Synthesis

Based on your exploration, write a brief summary (3-5 bullet points) of the most important findings from this dataset.

*Your findings here:*

1.
2.
3.
4.
5.

### Exercise 6.2: Data Limitations

What are the key limitations of this dataset that affect what questions we can answer?

*Your answer here:*



### Exercise 6.3: Question Classification

For each of the following questions, identify which of the seven question types it represents and whether our data can answer it:

1. "What is the average oxygen saturation for COVID-suspected patients?"
2. "Would providing flu vaccines earlier reduce hospitalizations?"
3. "Can we predict which patients will be admitted as inpatients based on their vital signs?"
4. "Is there a significant difference in mortality between age groups?"

*Your answers here:*

1. Question type: _____ | Data support: _____
2. Question type: _____ | Data support: _____
3. Question type: _____ | Data support: _____
4. Question type: _____ | Data support: _____

### Exercise 6.4: Next Steps

Based on your EDA, what would be your recommended next steps? Choose from:
- Communication and stakeholder alignment
- Modeling preparation
- Data and process redesign
- Goal refinement

Explain your reasoning.

*Your answer here:*



---
## Bonus Exercises

### Bonus 1: Predictive Model Preparation

Prepare features for a model that predicts inpatient admission based on vital signs and demographics.

1. Select relevant features
2. Handle missing values
3. Create the target variable
4. Check class balance

In [ ]:
# YOUR CODE HERE


### Bonus 2: Condition Co-occurrence

Create a co-occurrence matrix showing which conditions tend to appear together.

In [ ]:
# YOUR CODE HERE


### Bonus 3: Patient Journey Visualization

For patients with multiple visits, visualize their journey over time (conditions and visit types).

In [ ]:
# YOUR CODE HERE


---
## Summary: Key EDA Concepts

### The EDA Lifecycle
1. Clarify the analytical goal
2. Understand data provenance, structure, and integrity
3. Explore (distributional, relational, structural, comparative)
4. Interpret findings and refine hypotheses
5. Translate results into next steps

### The Seven Question Types
1. Descriptive — What happened?
2. Exploratory — What patterns exist?
3. Inferential — Does this generalize?
4. Predictive — What will happen?
5. Prescriptive — What should we do?
6. Causal — What if we intervene?
7. Mechanistic — What process produces this?

### The Four Exploration Dimensions
1. **Distributional** — Individual variable shape, outliers, missingness
2. **Relational** — Correlations, interactions between variables
3. **Structural** — Temporal, hierarchical, sequential patterns
4. **Comparative** — Differences across groups and time periods

### The Seven Core Principles
1. Let the data surprise you
2. Multiple encodings, multiple perspectives
3. Segment early, segment often
4. Explicitly check assumptions
5. Expect heterogeneity and drift
6. Distinguish signal from artifact
7. Document hypotheses and alternative explanations